In [1]:
import time

from numpy import interp
import numpy as np

from bokeh.io import show, output_notebook, push_notebook
from bokeh.io import curdoc 
from bokeh.palettes import *
from bokeh.resources import INLINE
from bokeh.plotting import figure
from bokeh.models import Range1d, LabelSet, Label, ColumnDataSource
from bokeh.palettes import Blues8, BrBG10, Pastel1_9, Spectral10, Category20c
from bokeh.models import Range1d

from ipywidgets import interact, interactive, fixed, interact_manual

from IPython.display import display
import ipywidgets as widgets

from bokeh.layouts import widgetbox
from bokeh.models.widgets import Button

import matplotlib as mpl
import matplotlib.cm as cm

import serial
import math
import struct
import binascii

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
'''
From the Grid Eye demo - https://eewiki.net/display/projects/Panasonic+GridEYE+Breakout+Board+and+GUI

'*' \
'*'  >-- Packet start designator
'*' /
Thermistor High Byte
Thermistor Low Byte
64 High Byte, Low Byte pairs

'''

"\nFrom the Grid Eye demo - https://eewiki.net/display/projects/Panasonic+GridEYE+Breakout+Board+and+GUI\n\n'*' '*'  >-- Packet start designator\n'*' /\nThermistor High Byte\nThermistor Low Byte\n64 High Byte, Low Byte pairs\n\n"

In [4]:
def floatRgb(mag, cmin, cmax):
       """
       Return a tuple of floats between 0 and 1 for the red, green and
       blue amplitudes.
       """

       try:
              # normalize to [0,1]
              x = float(mag-cmin)/float(cmax-cmin)
       except:
              # cmax = cmin
              x = 0.5
       blue = min((max((4*(0.75-x), 0.)), 1.))
       red  = min((max((4*(x-0.25), 0.)), 1.))
       green= min((max((4*math.fabs(x-0.5)-1., 0.)), 1.))
       return (red, green, blue)
    
def strRgb(mag, cmin, cmax):
       """
       Return a tuple of strings to be used in Tk plots.
       """

       red, green, blue = floatRgb(mag, cmin, cmax)       
       return "#%02x%02x%02x" % (red*255, green*255, blue*255)

def rgb(mag, cmin, cmax):
       """
       Return a tuple of integers to be used in AWT/Java plots.
       """

       red, green, blue = floatRgb(mag, cmin, cmax)
       return (int(red*255), int(green*255), int(blue*255))

def htmlRgb(mag, cmin, cmax):
       """
       Return a tuple of strings to be used in HTML documents.
       """
       return "#%02x%02x%02x"%rgb(mag, cmin, cmax)


In [5]:
ser = serial.Serial('/dev/ttyUSB1', 115200)  # open serial port
print(ser.name) 
ser.close()
ser.open()

/dev/ttyUSB1


In [6]:
#Define Varible
squares = []

#Create grid pattern
val = 0
xValsData = []
yValsData = []
blankData = []

for i in range(0,8):
    for j in range(0,8):
        
        xValsData.append(i)
        yValsData.append(j)
        blankData.append(float((val * 1.0 / 64.0)))
        
        val = val + 1
        j+=1
    j+=1
    i=0

data = {'x_values' : xValsData,
        'y_values' : yValsData,
        'dataColor' : blankData,
        'dataDecimal' : ['0', '1', '2', '3', '4', '5', '6', '7', 
                         '0', '1', '2', '3', '4', '5', '6', '7', 
                         '0', '1', '2', '3', '4', '5', '6', '7',
                         '0', '1', '2', '3', '4', '5', '6', '7',
                         '0', '1', '2', '3', '4', '5', '6', '7',
                         '0', '1', '2', '3', '4', '5', '6', '7',
                         '0', '1', '2', '3', '4', '5', '6', '7',
                         '0', '1', '2', '3', '4', '5', '6', '7']
}

source = ColumnDataSource(data=data)

In [7]:
def refreshSensor():
    # Write * to start streaming and ~ to stop
    ser.write('~')
    time.sleep(0.1)
    ser.write('*')
    time.sleep(0.1)

    #Find start of frame
    startFound = False
    bytesRead = 0

    thermistorH = 0
    thermistorL = 0
    dataArray = []

    while(startFound == False):
        temp = ser.read(1)
        if temp == '*':
            startFound = True
            print "Start Found"

            #Read next two *
            ser.read(2)

            #Read data
            thermistor = ser.read(2)

            for i in range(0, 64):
                dataArray.append( ser.read(2) )

    #Stop from getting more data
    startFound = False

    #print thermistor
    #print type(thermistor)

    #print binascii.hexlify(thermistor)
    #print type(binascii.hexlify(thermistor))

    thermistorHex = binascii.hexlify(thermistor)

    #print struct.unpack("<h", thermistor )[0] * 0.0625
    thermistor = struct.unpack("<h", thermistor )[0] * 0.0625

    dataDecimal = []
    for i in range(0, len(dataArray)):
        dataDecimal.append( struct.unpack("<h", dataArray[i])[0] * 0.25 )
        #print struct.unpack("<h", dataArray[i])[0] * 0.25

    dataColor = []

    minVal = (min(dataDecimal)) * 1.0
    print minVal
    maxVal = (max(dataDecimal)) * 1.0
    print maxVal

    for i in range(0, len(dataDecimal)):
        temp = struct.unpack("<h", dataArray[i])[0] * 0.25
        #print struct.unpack("<h", dataArray[i])[0] * 0.25

        dataColor.append( strRgb(temp, minVal, maxVal) )
        #print strRgb(temp, 0.0, 100.0)
    
    #Convert DataDecimal to str
    dataDecimal = map(str, dataDecimal)
    
    return dataColor, dataDecimal

In [8]:
def createPlot(_plt, _dataColor, _dataDecimal):

    _plt.square(x='x_values', y='y_values', color='dataColor', source=source, fill_alpha=1.0, size=64, line_color='black', line_alpha=1)
    
    labels = LabelSet(x='x_values', y='y_values', text='dataDecimal', source = source, level='glyph', x_offset=-17, y_offset=-10, render_mode='canvas')
    _plt.add_layout(labels)
        
    left, right, bottom, top = -2, 8, -2, 8
    _plt.x_range=Range1d(left, right)
    _plt.y_range=Range1d(bottom, top)
    _plt.title.text = 'Grid Eye'
    _plt.title.align = 'center'
    _plt.width=65
    _plt.height=65

    _plt.title.text_font_size = '22px'


In [15]:
def on_button_clicked(b):
    print("Button clicked.")
    dataColor, dataDecimal = refreshSensor()
    
    #Update Datasource
    data['dataColor'] = dataColor
    data['dataDecimal'] = dataDecimal
    source = ColumnDataSource(data=data)
    
    push_notebook(handle=handleT)
    
    print "Done updating"


In [13]:
#Get init data
dataColor, dataDecimal = refreshSensor()

#Update Datasource
data['dataColor'] = dataColor
data['dataDecimal'] = dataDecimal
source = ColumnDataSource(data=data)

pltA = figure(responsive=True, width=800, height=300, toolbar_location="above", tools='tap, box_zoom, pan, undo, crosshair, hover, reset, wheel_zoom, save')
createPlot(pltA, dataColor, dataDecimal)

#Show init plot
handleT = show(pltA, notebook_handle=True)

Start Found
18.25
27.5


In [16]:
#Create Button
button = widgets.Button(description="Click Me!")
display(button)

button.on_click(on_button_clicked)

Button(description=u'Click Me!', style=ButtonStyle())

Button clicked.
Start Found
18.25
26.25
Done updating
